In [16]:
# Store the csv as a df
import pandas as pd

df = pd.read_csv('data/landmarks.csv')
print(df.head())

   pose_id    x_nose     y_nose      z_nose  x_left_eye_inner  \
0        0 -5.889507 -57.637520  -45.019750         -4.656085   
1        1 -4.255504 -62.935925 -128.907500         -2.977403   
2        2 -2.878917 -61.709988 -137.453340         -1.619050   
3        3 -4.242575 -60.371220 -135.094830         -3.118133   
4        4 -0.805543 -56.178570  -41.124413         -0.055174   

   y_left_eye_inner  z_left_eye_inner  x_left_eye  y_left_eye  z_left_eye  \
0        -62.832863        -44.571823   -3.302626  -63.386856  -44.567863   
1        -67.035990       -124.258545   -2.215265  -67.198250 -124.263240   
2        -65.693750       -132.181660   -0.785822  -65.814340 -132.184070   
3        -64.416000       -129.995930   -2.369744  -64.603290 -130.003400   
4        -58.501305        -37.938560    0.456936  -58.473960  -37.954430   

   ...  z_left_heel  x_right_heel  y_right_heel  z_right_heel  \
0  ...    56.852562     -0.842025     35.037060     50.565020   
1  ...   -14.129

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [4]:
# Select features and target
X = df.drop(['pose_id'], axis=1)
y = df['pose_id']

In [12]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [14]:
y_encoded

array([   0,    1,    2, ..., 1369, 1370, 1371], dtype=int64)

In [15]:
print(y.shape)
y_categorical
print(y_categorical.shape)

(1372,)
(1372, 1372)


In [17]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Normalize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [1]:
X_train_scaled[0]

NameError: name 'X_train_scaled' is not defined

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [20]:
history = model.fit(X_train_scaled, y_train,
                    validation_split=0.2,
                    epochs=100,
                    batch_size=32)

Epoch 1/100
28/28 [==============================] - 0s 7ms/step - loss: 0.6581 - accuracy: 0.7719 - val_loss: 0.7256 - val_accuracy: 0.7545
Epoch 2/100
28/28 [==============================] - 0s 3ms/step - loss: 0.7074 - accuracy: 0.7423 - val_loss: 0.7267 - val_accuracy: 0.7682
Epoch 3/100
28/28 [==============================] - 0s 3ms/step - loss: 0.6865 - accuracy: 0.7526 - val_loss: 0.7379 - val_accuracy: 0.7773
Epoch 4/100
28/28 [==============================] - 0s 3ms/step - loss: 0.6823 - accuracy: 0.7583 - val_loss: 0.7034 - val_accuracy: 0.8000
Epoch 5/100
28/28 [==============================] - 0s 5ms/step - loss: 0.6712 - accuracy: 0.7594 - val_loss: 0.7272 - val_accuracy: 0.7636
Epoch 6/100
28/28 [==============================] - 0s 4ms/step - loss: 0.6817 - accuracy: 0.7605 - val_loss: 0.7382 - val_accuracy: 0.7818
Epoch 7/100
28/28 [==============================] - 0s 3ms/step - loss: 0.6800 - accuracy: 0.7514 - val_loss: 0.7466 - val_accuracy: 0.7727
Epoch 8/100
2

In [21]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

9/9 [==============================] - 0s 2ms/step - loss: 0.5933 - accuracy: 0.8509
Test Loss: 0.593292772769928, Test Accuracy: 0.8509091138839722


In [24]:
import joblib

model.save('saves/exercise_recognition_model.keras')
joblib.dump(label_encoder, 'saves/label_encoder.joblib')
joblib.dump(scaler, 'saves/scaler.save')

['saves/scaler.save']